In [1]:
# Recipe Database
# repo = "https://raw.githubusercontent.com/jakevdp/open-recipe-data/master"
# !cd data && curl -O {repo}/recipeitems.json.gz
# !gunzip data/recipeitems.json.gz

path = "/__PROJECTS/aa_CommonData/ml/"
import numpy as np 
import pandas as pd 

In [2]:
recipes = pd.read_json(path+"recipeitems.json",lines=True)
recipes.shape

(173278, 17)

In [3]:
# тестируем
recipes.iloc[0]

_id                                {'$oid': '5160756b96cc62079cc2db15'}
name                                    Drop Biscuits and Sausage Gravy
ingredients           Biscuits\n3 cups All-purpose Flour\n2 Tablespo...
url                   http://thepioneerwoman.com/cooking/2013/03/dro...
image                 http://static.thepioneerwoman.com/cooking/file...
ts                                             {'$date': 1365276011104}
cookTime                                                          PT30M
source                                                  thepioneerwoman
recipeYield                                                          12
datePublished                                                2013-03-11
prepTime                                                          PT10M
description           Late Saturday afternoon, after Marlboro Man ha...
totalTime                                                           NaN
creator                                                         

In [5]:
# для начала получим общую информацию:
# мы её получаем по списку ингридиентов 
recipes.ingredients.str.len().describe()

count    173278.000000
mean        244.617926
std         146.705285
min           0.000000
25%         147.000000
50%         221.000000
75%         314.000000
max        9067.000000
Name: ingredients, dtype: float64

In [6]:
recipes.name[np.argmax(recipes.ingredients.str.len())]

'Carrot Pineapple Spice &amp; Brownie Layer Cake with Whipped Cream &amp; Cream Cheese Frosting and Marzipan Carrots'

In [7]:
# заметьте поиск по маске с включением букв 
recipes.description.str.contains("[Bb]reakfast").sum()

3524

In [9]:
(
    recipes.ingredients.str.contains("[Cc]innamon").sum(),
    recipes.ingredients.str.contains("[Cc]inamon").sum(),
)

(10526, 11)

In [10]:
# A Simple Recipe Recommender
spice_list = [
    "salt","pepper","oregano","sage","parsley",
    "rosemary","tarragon","thyme","paprika","cumin"
]

In [11]:
import re 
spice_df = pd.DataFrame({
    spice:
    recipes.ingredients.str.contains(spice,re.IGNORECASE) for spice in spice_list
})
spice_df.head()

,salt,pepper,oregano,sage,parsley,rosemary,tarragon,thyme,paprika,cumin
0,False,False,False,True,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,True,True,False,False,False,False,False,False,False,True
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False


In [15]:
# как видим даже небольшая ошибка в query приводит не к возврату null 
# а к выбросу исключения:
try:
    selection = spice_df.query('parsley & paprika & terragon')
    len(selection)
except:
    print("wrong query")
    
selection = spice_df.query('parsley & paprika & tarragon')
len(selection)

wrong query


10

In [16]:
recipes.name[selection.index]

2069      All cremat with a Little Gem, dandelion and wa...
74964                         Lobster with Thermidor butter
93768      Burton's Southern Fried Chicken with White Gravy
113926                     Mijo's Slow Cooker Shredded Beef
137686                     Asparagus Soup with Poached Eggs
140530                                 Fried Oyster Po’boys
158475                Lamb shank tagine with herb tabbouleh
158486                 Southern fried chicken in buttermilk
163175            Fried Chicken Sliders with Pickles + Slaw
165243                        Bar Tartine Cauliflower Salad
Name: name, dtype: object